## This demo provides an implementation of sufrface coil intensity correction (SCC) for data collected on a Siemens scanner.
### Make sure that you have cloned the [SCC repository](https://github.com/OSU-MR/SCC) from GitHub and are executing brightness_correction_demo_ipynb from the SCC folder.
### Before you run the demo, make sure that the your environment is setup properly. To do that, execute these commands in the terminal. Then, make sure that the SCC environment is selected for this code to run.
* conda create --name SCC python=3.8
* conda activate SCC
* conda install jupyterlab
* conda install ipykernel
* python -m ipykernel install --user --name=SCIC


### The following cell will install [Twix Tools](https://github.com/OSU-MR/Python_read_Siemens_rawdata) and then replace twix_map.py with the one modified by Dr. Chong Chen (GitHub ID: MRIOSU)
* Twix Tools are needed to read the raw datafile from Siemens scanners

In [3]:
from helper_functions.download_data import download_file_from_figshare, install_twixtools 
install_twixtools()

Extracting the zip file...
Replacing twix_map.py...
Sufficient numpy version is already installed.
Installing twixtools...
Processing /Users/ahmad.46/Library/CloudStorage/OneDrive-TheOhioStateUniversity/software/github-repos/SCC/twixtools-master
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for twixtools: filename=twixtools-1.0-py3-none-any.whl size=51389 sha256=58cbc85ec84ce213c8ab36dae6ccd016206c8dde8851c6c174c61a859d2e29ec
  Stored in directory: /Users/ahmad.46/Library/Caches/pip/wheels/46/e8/b9/a19a7d980fc65bd8322e2a43100810615748a8b7cdc5061903
Successfully built twixtools
  Attempting uninstall: twixtools
    Found existing installation: twixtools 1.0
    Uninstalling twixtools-1.0:
      Successfully uninstalled twixtools-1.0
Removing the zip file and the extracted directory...
Installing numpy==1.23.4...
Installing sigpy==0.1.25...
Installing matplotlib==3.7.2...
Installation of missing packages complete!
In

<!-- # 2. Organize the folders of your datasets
# **The sturcture of your dataset folders should be like this:**
<style>
code {
    font-size: 15px;
}
</style>
```markdown


base_dir-----input_folder-----folder name of your datasets_1
          |                |--folder name of your datasets_2
          |                |              ...
          |		        --folder name of your datasets_n
          |
          |
          | (folders below will be automatically created)
          ---output_folder------correction map folder of your datasets_1    
			     |--correction map folder of your datasets_2         
			     |                ...                 
                 ---correction map folder of your datasets_n

``` -->

### This cell will automatically download a raw data file that we have uploaded on [figshare](figshare.com). Be patient, downloading the file may take a few minutes.
* The downloaded file is saved in SCC/data/rawdata/demo folder while the results are saved on SCC/data/results/demo
* The link 'url_2CH' has a series of images from a single slice (620 MB). The link 'url_SAX' has series of images from multiple slices (5.2 GB)

In [4]:
import os
from helper_functions.download_data import download_file_from_figshare
# Path: ./data
#create base directory
os.makedirs('./data', exist_ok=True)

#create subdirectories
os.makedirs('./data/rawdata', exist_ok=True)
os.makedirs('./data/rawdata/demo', exist_ok=True)

#download the data
# saving_path = './data/rawdata/'
saving_path = './data/rawdata/demo/'
url_2CH = "https://figshare.com/ndownloader/files/41881158" # image collected in two-chambers view
# url_SAX = "https://figshare.com/ndownloader/files/41915115" # image stack collected in short-axis view

print("Demo datasets are being downloaded, please wait......")
download_file_from_figshare(saving_path, url_2CH) # May take a couple of minutes to download
# download_file_from_figshare(saving_path,url_SAX) # May take tens of minutes to download


# if we are using free version of Google Colab then you may not want to download the multi-slice short-axis (SAX) data because of it's large size
# if 'google.colab' not in str(get_ipython()):
    # download_file_from_figshare(saving_path,url_SAX) # May take tens of minutes to download

base_dir = "./data"
input_folder = "rawdata"
input_subfolders = ['demo'] #set this to None, if you want to go through all the subfolders inside input_folder
output_folder = "results"

Demo datasets are being downloaded, please wait......
File saved as meas_MID00091_FID54254_DE_TRUFI_2CH_MOCO_30avg.dat


<!-- # 3. Import the functions and define the folders -->

In [ ]:

# base_dir = "./data"
# input_folder = "rawdata"
# output_folder = "SCC_results"


### The data that we have provided is fully sampled. So, no reconstruction is required other than coil-combiniging. If your data is undersampled, you could call your own reconstruction routine from the function 'CustomProcedure'.

In [5]:
from helper_functions.brightness_correction import getting_and_saving_correction_map, create_and_start_threadings, displaying_results
import numpy as np
def rms_comb(sig, axis=1): # sum-of-square, square root
    return np.sqrt(np.sum(abs(sig)**2, axis))

def CustomProcedure(img_body_coils, img_surface_coils, ksp, ref_padded, noise_kspace, dim_info_noise):

    # print('img_body_coils.shape', img_body_coils.shape)
    # print('img_surface_coils.shape', img_surface_coils.shape)
    # print('ksp.shape', ksp.shape)
    # print('ref_padded.shape', ref_padded.shape)
    # print('noise_kspace.shape', noise_kspace.shape)
    # print('dim_info_noise.shape', dim_info_noise.shape)

    img_body_coils = rms_comb(img_body_coils,0)
    img_surface_coils = rms_comb(img_surface_coils,0)

    #set Stop to True to stop the program for debugging
    Stop = False
    assert not Stop, "Stopped for debugging"
    
    return img_body_coils, img_surface_coils , ksp, ref_padded

/Users/ahmad.46/anaconda3/envs/SCC/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Apply SCC to correct intensity

In [7]:
num_thread = 1 # define the number of threads
threads = create_and_start_threadings(num_thread, getting_and_saving_correction_map, base_dir, 
                                      input_folder, output_folder, input_subfolders, 
                                      auto_rotation='LGE',debug = True,
                                      apply_correction_during_sense_recon=False,CustomProcedure=CustomProcedure)

import time

while True:
    alive_flag = False
    for i in range(len(threads)):
        alive_flag = alive_flag or threads[i].is_alive()
    if alive_flag is False:
        break
    time.sleep(0.5)
print("All threads have finished their jobs!")

Thread 0 has been created! please wait until it finishes!
Software version: VD/VE (!?)

Scan  0


100%|██████████| 56.2M/56.2M [00:00<00:00, 506MB/s]


Scan  1


100%|██████████| 535M/535M [00:00<00:00, 917MB/s] 



number of separate scans (multi-raid): 2
dim_info_org
 ['Set', 'Ave', 'Lin', 'Cha', 'Col'] 
 (2, 30, 150, 30, 512)
multiple sets
original data shape: (2, 32, 32, 30, 128)
['Set', 'Par', 'Lin', 'Cha', 'Col']
(2, 32, 32, 30, 128)
1 1
arranged data shape (128, 32, 32, 30, 2, 1, 1)
['RO', 'E1', 'E2', 'Cha', 'Set', 'S', 'Sli']
3d image shape: (128, 32, 32, 30, 2)
Dimension 'Sli' not found, skipping.
custom procedure found and used!
apply_correction_during_sense_recon:  False


# 6. For displaying the result, use **displaying_results()** function like this:

In [ ]:
sli_idx = 0 #changing this number will show different slices
output_folder_names = ['dataset_demo']
output_folder = "correction_maps_demo"
displaying_results(base_dir=base_dir, input_folder=input_folder,
                   output_folder=output_folder, folder_names=output_folder_names, sli_idx=sli_idx,
                   fig_h=9, debug=False) #set debug = True to display more information


# 7. Set **apply_correction_during_sense_recon = True** to apply the correction during the sense reconstruction

In [ ]:
num_thread = 1 # define the number of threads
threads = create_and_start_threadings(num_thread, getting_and_saving_correction_map, base_dir, 
                                      input_folder, output_folder, input_folder_names, 
                                      auto_rotation='LGE',debug = True,
                                      apply_correction_during_sense_recon=True,CustomProcedure=CustomProcedure)

while True:
    alive_flag = False
    for i in range(len(threads)):
        alive_flag = alive_flag or threads[i].is_alive()
    if alive_flag is False:
        break
    time.sleep(0.5)
print("All threads have finished their jobs!")

# 8. Set **debug = True** to display the all results generated above

In [ ]:
output_folder_names = ['dataset_demo']
output_folder = "correction_maps_demo"
displaying_results(base_dir=base_dir, input_folder=input_folder,
                   output_folder=output_folder, folder_names=output_folder_names, sli_idx=0,
                   fig_h=9, debug=True) #set debug = True to display more information